In [154]:
import os
from pathlib import Path

import pandas as pd
import numpy as np

path_root = Path("../../data/data_chapters/")

In [155]:
# Recuperamos el dataset base ya filtrado
dataset_df = pd.read_csv("../../data/dataset_4.csv", usecols = ["uuid","type","demography"]) # index_col=0)
dataset_df.columns

Index(['demography', 'type', 'uuid'], dtype='object')

In [156]:
# nos quedamos con las columnas idetificadoras para el cruce
data_search_df = dataset_df[["uuid","type","demography"]]
#data_search_df

In [157]:
# Preparamos el path root de los capitulos
os.chdir(path_root)
print(os.getcwd())

c:\Users\usuario\Documents\GitHub\EDA_manga_online\src\data\data_chapters


In [158]:
# Recuperamos todos los sub folders de capitulos
folders_path = []
for i in os.listdir():
    if '.' not in i:
        folders_path.append(os.path.join(path_root, i))

In [159]:
# Recuperamos todos los path de los fichero de capitulos
files = []
for folder_path in folders_path:
    tree_file = os.walk(folder_path)
    for data_file in tree_file:
        tree_folder = data_file[0]
        files.extend([os.path.join(tree_folder, file) for file in data_file[2]])

len(files)

19084

In [160]:
# Creamos el dataframe base
file_uuid = files[0].split("\\")[-1].split(".")[0]
chapter_df = pd.read_csv(files[0], usecols = ["date_upload"])
key = dataset_df[dataset_df["uuid"] == file_uuid]
chapter_df["uuid"] = file_uuid
dataset_chapters_df = pd.merge(chapter_df, key)

In [161]:
# Ojo q tarda 3 min o mas
# Cargamos el resto de los datos
no_upload_date = 0
for file in files[1:]: 
    try:
        chapter_df = pd.read_csv(file, usecols = ["date_upload"])  
    except ValueError as err: # ignoramos los que no tienen fecha de subida
        no_upload_date += 1
    else:
        file_uuid = file.split("\\")[-1].split(".")[0]
        key = dataset_df[dataset_df["uuid"] == file_uuid]
        chapter_df["uuid"] = file_uuid
        result = pd.merge(chapter_df, key)
        dataset_chapters_df = pd.concat([dataset_chapters_df, result])

print(no_upload_date)


22


In [162]:
# Limpiamos un espacio al inicio del campo
dataset_chapters_df["date_upload"] = [row.replace(" ", "") for row in dataset_chapters_df["date_upload"]]

In [163]:
# Extraemos los registros con mas de una subida por capitulo
dataset_multy_chapters_df = dataset_chapters_df[dataset_chapters_df["date_upload"].str.len() > 10].copy()

In [164]:
# Dejamos una sola fecha para los capitulos que tienen mas de una subida. ###
print(len(dataset_chapters_df))
dataset_chapters_df["date_upload"] = dataset_chapters_df["date_upload"].apply(lambda data : data.split(",")[0] if len(data) > 10 else data)
print(len(dataset_chapters_df))

575336
575336


In [165]:
# Eliminados las fechas errornes = '-0001-11-30'
dataset_chapters_df = dataset_chapters_df[dataset_chapters_df.date_upload.str.len() == 10]
print(len(dataset_chapters_df))

575192


In [166]:
# Desglosamos los capitulos con subidas multiples a un resgitro por subida
list_chapters = []
print(len(dataset_multy_chapters_df))
for index, row in dataset_multy_chapters_df.iterrows():
    list_date_upload = row["date_upload"].split(",")[1:]
    for date in list_date_upload:
        list_chapters.append({"date_upload": date, 
                              "uuid": row["uuid"], 
                              "demography": row["demography"],
                              "type": row["type"]})
        
print(len(list_chapters))

72936
86124


In [167]:
dataset_multy_chapters = pd.DataFrame(list_chapters)
print(len(dataset_multy_chapters))

86124


In [168]:
dataset_chapters_df.to_csv("../../data/dataset_chapters.csv")

In [169]:
dataset_chapters_df.to_csv("../../data/dataset_multy_chapters.csv")